# T09 : Évaluation Comparative Robuste (ML vs Règles vs RL V8)

## Objectif
Ce notebook final vise à comparer les performances de trois approches de trading sur deux jeux de données inédits :
- **Test Set (2024)** : Année de test standard.
- **Forward Test (2025)** : Année de validation "Out-of-Sample" la plus récente.

### Stratégies Comparées
1.  **Approche Règles (Rule-Based)** : Stratégie classique EMA + RSI.
2.  **Approche Machine Learning (ML)** : Random Forest (Classification de la direction).
3.  **Approche Reinforcement Learning (RL)** : Modèle PPO optimisé (V8) chargé depuis `models/v8/`.

## Méthodologie
- **Données** : GBP/USD M15 (2022-2025).
- **Train** : 2022-2023 (Pour ML).
- **Test** : 2024.
- **Validation/Forward** : 2025.
- **Coûts** : Spread + Commission simulés (0.00015 ou 1.5 pips).
- **Métriques** : Sharpe Ratio, Max Drawdown, Profit Factor, Win Rate.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import os
import sys

# Ajout du root au path pour importer les modules du projet
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Import RL libs (si disponibles)
try:
    from stable_baselines3 import PPO
    from training.trading_env_v8 import TradingEnvV8
    RL_AVAILABLE = True
except ImportError:
    print("Stable-Baselines3 non installé. Le RL ne sera pas exécuté.")
    RL_AVAILABLE = False

# Configuration visuelle
plt.style.use('seaborn-v0_8-pastel')
sns.set_theme(style="whitegrid")
COLORS = {'Train': '#EAE7DC', 'Val': '#D8C3A5', 'Test': '#8E8D8A', 'Profit': '#4E8D7C', 'Loss': '#E85A4F'}

DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'm15')
MODEL_PATH_V8 = os.path.join(PROJECT_ROOT, 'models', 'v8', 'ppo_trading_best.zip')
print(f"Data Directory: {DATA_DIR}")
print(f"Model V8 Path : {MODEL_PATH_V8}")

## 1. Chargement et Feature Engineering

In [ ]:
def add_technical_features(df):
    df = df.copy()
    c = df['close_15m']
    h = df['high_15m']
    l = df['low_15m']
    
    # --- Basic Features (pour Règles + ML) ---
    # EMA
    df['ema_20'] = c.ewm(span=20, adjust=False).mean()
    df['ema_50'] = c.ewm(span=50, adjust=False).mean()
    df['ema_200'] = c.ewm(span=200, adjust=False).mean()
    
    # RSI
    delta = c.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    # --- Advanced Features (pour ML) ---
    # ATR
    tr = pd.concat([h-l, abs(h-c.shift()), abs(l-c.shift())], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    
    # ADX (approx simple)
    plus_dm = h.diff()
    minus_dm = -l.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0
    
    tr14 = tr.rolling(14).sum()
    plus_di = 100 * (plus_dm.ewm(alpha=1/14).mean() / (df['atr_14'] + 1e-10))
    minus_di = 100 * (minus_dm.ewm(alpha=1/14).mean() / (df['atr_14'] + 1e-10))
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di + 1e-10)) * 100
    df['adx_14'] = dx.rolling(14).mean()
    
    # Returns & Volatility
    df['return_15m'] = c.pct_change()
    df['rolling_std_20'] = c.rolling(20).std()
    
    # Target for ML (Next Close > Current Close)
    df['target'] = (c.shift(-1) > c).astype(int)
    
    df.dropna(inplace=True)
    return df

def load_data(data_dir):
    files = ['GBPUSD_M15_2022.csv', 'GBPUSD_M15_2023.csv', 'GBPUSD_M15_2024.csv', 'GBPUSD_M15_2025.csv']
    dfs = []
    for f in files:
        path = os.path.join(data_dir, f)
        if os.path.exists(path):
            print(f"Loading {f}...")
            df_year = pd.read_csv(path, parse_dates=['timestamp'], index_col='timestamp')
            dfs.append(df_year)
        else:
            print(f"Warning: Fichier manquant {path}")
            
    if not dfs:
        raise FileNotFoundError(f"Aucun fichier de données trouvé dans {data_dir}")
        
    df = pd.concat(dfs)
    df.sort_index(inplace=True)
    
    # Add Features
    df = add_technical_features(df)
    
    return df

df_full = load_data(DATA_DIR)
print(f"Données chargées & Features calculées (Total): {df_full.shape}")

# Split par année
train_data = df_full.loc['2022']
val_data = df_full.loc['2023']
test_data_2024 = df_full.loc['2024']
test_data_2025 = df_full.loc['2025'] if '2025' in df_full.index.year.astype(str) else pd.DataFrame()

print(f"Train (2022)      : {train_data.shape}")
print(f"Val (2023)        : {val_data.shape}")
print(f"Test (2024)       : {test_data_2024.shape}")
print(f"Forward (2025)    : {test_data_2025.shape}")

## 2. Moteur de Backtest Vectorisé

In [ ]:
class Backtester:
    def __init__(self, data, strategy_name, initial_capital=10000, transaction_cost=0.00015):
        self.data = data.copy()
        self.strategy_name = strategy_name
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.results = None

    def run(self, signals):
        """
        signals: pd.Series avec index timestamp et valeurs {1 (Buy), -1 (Sell), 0 (Cash/Hold)}
        """
        # Alignement des signaux
        self.data['signal'] = signals
        self.data['signal'] = self.data['signal'].shift(1) # On trade à l'ouverture suivante
        self.data['signal'].fillna(0, inplace=True)
        
        market_returns = self.data['return_15m']
        
        # Coûts : à chaque changement de position
        trades = self.data['signal'].diff().abs()
        costs = trades * self.transaction_cost
        
        strategy_returns = (self.data['signal'] * market_returns) - costs
        
        # Capital Curve
        self.data['strategy_returns'] = strategy_returns
        self.data['equity'] = (1 + strategy_returns).cumprod() * self.initial_capital
        self.data['drawdown'] = self.data['equity'] / self.data['equity'].cummax() - 1
        
        return self.data['equity']

    def metrics(self):
        total_return = (self.data['equity'].iloc[-1] / self.initial_capital) - 1
        # Annualized Sharpe (assuming risk-free rate = 0)
        std = self.data['strategy_returns'].std()
        sharpe = (self.data['strategy_returns'].mean() / std * np.sqrt(252 * 96)) if std > 0 else 0
        max_dd = self.data['drawdown'].min()
        
        # Profit Factor
        wins = self.data['strategy_returns'][self.data['strategy_returns'] > 0].sum()
        losses = self.data['strategy_returns'][self.data['strategy_returns'] < 0].sum()
        profit_factor = abs(wins / losses) if losses != 0 else np.inf
        
        return {
            'Strategy': self.strategy_name,
            'Total Return': f"{total_return:.2%}",
            'Sharpe Ratio': f"{sharpe:.2f}",
            'Max Drawdown': f"{max_dd:.2%}",
            'Profit Factor': f"{profit_factor:.2f}",
            'Final Equity': f"{self.data['equity'].iloc[-1]:.2f}"
        }

## 3. Stratégie 1 : Règles (EMA + RSI)

In [ ]:
def strategy_ema_rsi(df):
    signals = pd.Series(0, index=df.index)
    
    # Trend filter
    trend_bull = df['ema_20'] > df['ema_50']
    trend_bear = df['ema_20'] < df['ema_50']
    
    # Entries (Mean Reversion in Trend)
    long_entry = trend_bull & (df['rsi_14'] < 40)
    short_entry = trend_bear & (df['rsi_14'] > 60)
    
    signals[long_entry] = 1
    signals[short_entry] = -1
    
    # Forward fill (Hold position)
    signals = signals.replace(0, np.nan).ffill().fillna(0)
    
    return signals

## 4. Stratégie 2 : Machine Learning (Random Forest)

In [ ]:
# Features pour le ML
features_cols = ['rsi_14', 'ema_20', 'ema_50', 'atr_14', 'adx_14', 'return_15m', 'rolling_std_20']

# Standardisation (Fit sur 2022+2023)
scaler = StandardScaler()
X_train_full = pd.concat([train_data, val_data])
scaler.fit(X_train_full[features_cols])

X_train = scaler.transform(X_train_full[features_cols])
y_train = X_train_full['target']

# Entraînement
model_rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_rf.fit(X_train, y_train)

def get_ml_signals(df_test):
    if df_test.empty: return pd.Series()
    X_test = scaler.transform(df_test[features_cols])
    obs_pred = model_rf.predict(X_test)
    # Conversion 0/1 -> -1/1 (0 -> Short, 1 -> Long)
    return pd.Series(np.where(obs_pred == 1, 1, -1), index=df_test.index)

## 5. Stratégie 3 : Reinforcement Learning (V8)

In [ ]:
def get_rl_signals(df_test, year_label="2024"):
    if df_test.empty: return pd.Series()
    
    if not (RL_AVAILABLE and os.path.exists(MODEL_PATH_V8)):
        print(f"[RL {year_label}] Modèle introuvable.")
        return pd.Series(0, index=df_test.index)

    print(f"[RL {year_label}] Running Backtest V8...")
    model_rl = PPO.load(MODEL_PATH_V8)
    
    # Recharger les données brutes pour l'environnement (sans le pré-traitement du notebook)
    raw_file = os.path.join(DATA_DIR, f'GBPUSD_M15_{year_label}.csv')
    if not os.path.exists(raw_file):
        print(f"Fichier brut {raw_file} manquant.")
        return pd.Series(0, index=df_test.index)
        
    df_test_raw = pd.read_csv(raw_file)
    
    try:
        env_test = TradingEnvV8(
            df=df_test_raw,
            spread=0.00015,
            take_profit_pct=0.003,
            stop_loss_pct=0.002,
            max_hold=48,
            cooldown=4
        )
        
        obs, _ = env_test.reset()
        positions = []
        done = False
        
        while not done:
            action, _ = model_rl.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env_test.step(action)
            done = terminated or truncated
            positions.append(info['position'])
            
        return pd.Series(positions, index=df_test.index[:len(positions)])
        
    except Exception as e:
        print(f"Erreur Execution RL: {e}")
        return pd.Series(0, index=df_test.index)

## 6. Comparaison Finale

In [ ]:
def run_comparison(df_test, year_label):
    if df_test.empty:
        print(f"Pas de données pour {year_label}")
        return
        
    print(f"\n--- Lancements Backtests : {year_label} ---")
    results_list = []
    
    # 1. Règles EMA+RSI
    bt_rules = Backtester(df_test, "EMA + RSI")
    equity_rules = bt_rules.run(strategy_ema_rsi(df_test))
    results_list.append(bt_rules.metrics())
    
    # 2. Machine Learning RF
    bt_ml = Backtester(df_test, "Machine Learning (RF)")
    equity_ml = bt_ml.run(get_ml_signals(df_test))
    results_list.append(bt_ml.metrics())
    
    # 3. Reinforcement Learning V8
    bt_rl = Backtester(df_test, "RL PPO (V8)")
    equity_rl = bt_rl.run(get_rl_signals(df_test, year_label))
    results_list.append(bt_rl.metrics())
    
    # 4. Buy & Hold
    signals_bh = pd.Series(1, index=df_test.index)
    bt_bh = Backtester(df_test, "Buy & Hold (Baseline)")
    equity_bh = bt_bh.run(signals_bh)
    results_list.append(bt_bh.metrics())
    
    # --- Affichage Dataframe ---
    results_df = pd.DataFrame(results_list)
    print(f"### Tableau Comparatif ({year_label}) ###")
    display(results_df)
    
    # --- Plot ---
    plt.figure(figsize=(12, 6))
    plt.plot(equity_bh, label='Buy & Hold', color='gray', linestyle='--', alpha=0.5)
    plt.plot(equity_rules, label='EMA + RSI', color=COLORS['Train'])
    plt.plot(equity_ml, label='Random Forest', color=COLORS['Loss'])
    plt.plot(equity_rl, label='RL PPO V8', color=COLORS['Profit'], linewidth=2)
    
    plt.title(f"Comparaison des Stratégies sur {year_label}", fontsize=14)
    plt.ylabel("Capital ($)")
    plt.xlabel("Temps")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    save_path = f'../artifacts/T09_comparative_equity_{year_label}.png'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path)
    print(f"Graphique sauvegardé : {save_path}")
    plt.show()

In [ ]:
# Exécution sur 2024 (Test)
run_comparison(test_data_2024, "2024")

In [ ]:
# Exécution sur 2025 (Forward Test)
run_comparison(test_data_2025, "2025")